<a href="https://colab.research.google.com/github/eve-okorie16/testrep/blob/main/A_movie_recmmendation_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# movie recommendation system with cosine similarity

movies are recommended to users based on the type of movie(genre), the director of the movie, cast or even release year. i will build this system based on the genre and director

In [ ]:
import pandas as pd
import numpy as np


In [ ]:
df = pd.read_csv("movie analysis.csv")

In [ ]:
df.head(10)

,Unnamed: 0,Title,Release Date,Genre,Director,Budget,Revenue,Profit,Status,Release Year
0,0,The Devil Inside,2012-01-06,Horror,William Brent Bell,1000000,101800000.0,100800000.0,gain,2012
1,1,Red Tails,2012-01-20,Drama,Anthony Hemingway,58000000,50400000.0,-7600000.0,loss,2012
2,2,Declaration of War,2012-01-27,Romance,Valérie Donzelli,1500000,6500000.0,5000000.0,gain,2012
3,3,Man on a Ledge,2012-01-27,Action,Asger Leth,42000000,46200000.0,4200000.0,gain,2012
4,4,One for the Money,2012-01-27,Crime,Julie Anne Robinson,40000000,36900000.0,-3100000.0,loss,2012
5,5,The Grey,2012-01-27,Action,Joe Carnahan,25000000,77300000.0,52300000.0,gain,2012
6,6,Chronicle,2012-02-03,Sci-Fi,Josh Trank,12000000,126600000.0,114600000.0,gain,2012
7,7,The Woman in Black,2012-02-03,Horror,James Watkins,15000000,128500000.0,113500000.0,gain,2012
8,8,Journey 2: The Mysterious Island,2012-02-10,Adventure,Brad Peyton,79000000,335300000.0,256300000.0,gain,2012
9,9,Safe House,2012-02-10,Action,Daniel Espinosa,85000000,208100000.0,123100000.0,gain,2012


In [ ]:
df.columns

Index(['Unnamed: 0', 'Title', 'Release Date', 'Genre', 'Director', 'Budget',
       'Revenue', 'Profit', 'Status', 'Release Year'],
      dtype='object')

In [ ]:
df.info

<bound method DataFrame.info of      Unnamed: 0                     Title Release Date      Genre  \
0             0          The Devil Inside   2012-01-06     Horror   
1             1                 Red Tails   2012-01-20      Drama   
2             2        Declaration of War   2012-01-27    Romance   
3             3            Man on a Ledge   2012-01-27     Action   
4             4         One for the Money   2012-01-27      Crime   
..          ...                       ...          ...        ...   
503         503  Kubo and the Two Strings   2016-08-19    Fantasy   
504         504                  War Dogs   2016-08-19      Crime   
505         505             Don't Breathe   2016-08-26   Thriller   
506         506            Hands of Stone   2016-08-26  Biography   
507         507    Mechanic: Resurrection   2016-08-26     Action   

                Director    Budget      Revenue       Profit Status  \
0     William Brent Bell   1000000  101800000.0  100800000.0   gain 

we can see the info is not giving much information required about the data types so we check the data types

In [ ]:
df.dtypes

Unnamed: 0        int64
Title            object
Release Date     object
Genre            object
Director         object
Budget            int64
Revenue         float64
Profit          float64
Status           object
Release Year      int64
dtype: object

In [ ]:
#total number of duplicates in my dataframe
df.duplicated().sum()

0

dataframe has no duplicates so we move to drop the column unnmaed

In [ ]:
#the unmamed column is not needed hence will drop it
df.drop("Unnamed: 0", axis=1, inplace=True)

In [ ]:
df.head()

,Title,Release Date,Genre,Director,Budget,Revenue,Profit,Status,Release Year
0,The Devil Inside,2012-01-06,Horror,William Brent Bell,1000000,101800000.0,100800000.0,gain,2012
1,Red Tails,2012-01-20,Drama,Anthony Hemingway,58000000,50400000.0,-7600000.0,loss,2012
2,Declaration of War,2012-01-27,Romance,Valérie Donzelli,1500000,6500000.0,5000000.0,gain,2012
3,Man on a Ledge,2012-01-27,Action,Asger Leth,42000000,46200000.0,4200000.0,gain,2012
4,One for the Money,2012-01-27,Crime,Julie Anne Robinson,40000000,36900000.0,-3100000.0,loss,2012


In [ ]:
df.tail()

,Title,Release Date,Genre,Director,Budget,Revenue,Profit,Status,Release Year
503,Kubo and the Two Strings,2016-08-19,Fantasy,Travis Knight,60000000,27600000.0,-32400000.0,loss,2016
504,War Dogs,2016-08-19,Crime,Todd Phillips,40000000,42700000.0,2700000.0,gain,2016
505,Don't Breathe,2016-08-26,Thriller,Fede Alvarez,9900000,28300000.0,18400000.0,gain,2016
506,Hands of Stone,2016-08-26,Biography,Jonathan Jakubowicz,20000000,1700000.0,-18300000.0,loss,2016
507,Mechanic: Resurrection,2016-08-26,Action,Dennis Gansel,40000000,7500000.0,-32500000.0,loss,2016


we do a little data preprocessing by replacing any NaN value with a space to not get an error using a for loop

In [ ]:
features = ['Title', 'Genre', 'Director']
for feature in features:
    df[feature] = df[feature].fillna('')

we will combine all useful features by defining a function

In [ ]:
df['combined_features'] = df['Title']+" "+ df['Genre']+" "+ df['Director']
df['combined_features']

0          The Devil Inside Horror William Brent Bell
1                   Red Tails Drama Anthony Hemingway
2         Declaration of War Romance Valérie Donzelli
3                    Man on a Ledge Action Asger Leth
4         One for the Money Crime Julie Anne Robinson
                            ...                      
503    Kubo and the Two Strings Fantasy Travis Knight
504                      War Dogs Crime Todd Phillips
505               Don't Breathe Thriller Fede Alvarez
506      Hands of Stone Biography Jonathan Jakubowicz
507       Mechanic: Resurrection Action Dennis Gansel
Name: combined_features, Length: 508, dtype: object

There is no need to drop all other columns so we check the dataframe

In [ ]:
df.tail()

,Title,Release Date,Genre,Director,Budget,Revenue,Profit,Status,Release Year,combined_features
503,Kubo and the Two Strings,2016-08-19,Fantasy,Travis Knight,60000000,27600000.0,-32400000.0,loss,2016,Kubo and the Two Strings Fantasy Travis Knight
504,War Dogs,2016-08-19,Crime,Todd Phillips,40000000,42700000.0,2700000.0,gain,2016,War Dogs Crime Todd Phillips
505,Don't Breathe,2016-08-26,Thriller,Fede Alvarez,9900000,28300000.0,18400000.0,gain,2016,Don't Breathe Thriller Fede Alvarez
506,Hands of Stone,2016-08-26,Biography,Jonathan Jakubowicz,20000000,1700000.0,-18300000.0,loss,2016,Hands of Stone Biography Jonathan Jakubowicz
507,Mechanic: Resurrection,2016-08-26,Action,Dennis Gansel,40000000,7500000.0,-32500000.0,loss,2016,Mechanic: Resurrection Action Dennis Gansel


In [ ]:
#using feature extraction by importing sklearn feature extaction
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
CV = CountVectorizer()
count_matrix = CV.fit_transform(df['combined_features'])
print("Count Matrix:", count_matrix.toarray())

Count Matrix: [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [ ]:
cosine_sim = cosine_similarity(count_matrix)
cosine_sim

array([[1.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 1.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 1.        , ..., 0.        , 0.16666667,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.16666667, ..., 0.        , 1.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        1.        ]])

The closer the value of the cosine similarity close to one, the mre similar  the movies are and can grouped based on the features.

In [ ]:
cosine_sim.shape

(508, 508)

In [ ]:
indices = pd.Series(df.index, index=df['Title']).drop_duplicates()

In [ ]:
def get_recommendations(Title, cosine_sim=cosine_sim):
  idx = indices[Title]
  sim_scores = list(enumerate(cosine_sim[idx]))
  sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
  sim_scores = sim_scores[1:15]
  movie_indices = [i[0] for i in sim_scores]
  return df['Title'].iloc[movie_indices]

In [ ]:
get_recommendations("Man on a Ledge")

361                        Ant-Man
153                   Man of Steel
372        The Man from U.N.C.L.E.
51          The Amazing Spider-Man
124                  Dead Man Down
90     The Man with the Iron Fists
163                       R.I.P.D.
27                         Lockout
38                      Battleship
76                          Looper
79                         Taken 2
134                       Oblivion
164                          RED 2
171                         2 Guns
Name: Title, dtype: object

In [ ]:
get_recommendations("Don't Breathe")

133                Evil Dead
497                    Nerve
34                 The Raven
84     Paranormal Activity 4
165            The Conjuring
275            The Equalizer
333               The Gunman
368                 The Gift
433             The Revenant
198        A Most Wanted Man
202                 Non-Stop
263         The November Man
379          The Perfect Guy
457      10 Cloverfield Lane
Name: Title, dtype: object

Voila! my movie recommendation system. Thank you